In [1]:
import pandas as pd
import numpy as np

# collisions.csv

In [2]:
data_folder = '../CSV-2018/'

collisions = pd.read_csv(data_folder+'collisions2018.csv', error_bad_lines=False, low_memory=False)

In [3]:
collisions.columns

Index(['case_id', 'collision_date', 'collision_severity', 'collision_time',
       'county_city_location', 'hit_and_run', 'jurisdiction', 'lighting',
       'location_type', 'officer_id', 'pcf_violation',
       'pcf_violation_category', 'pcf_violation_subsection', 'population',
       'primary_collision_factor', 'process_date', 'ramp_intersection',
       'road_condition_1', 'road_condition_2', 'road_surface', 'tow_away',
       'type_of_collision', 'weather_1', 'weather_2'],
      dtype='object')

In [4]:
collisions.columns = ['case_id', 'col_date', 'col_severity', 'col_time',
       'county_city', 'hit_run', 'jurisdiction', 'lighting',
       'loc_type', 'officer_id', 'pcf_violation_code',
       'pcf_violation_category', 'subsection', 'population',
       'pcf_type', 'process_date', 'ramp_int',
       'road_con_1', 'road_con_2', 'road_surf', 'tow_away',
       'col_type', 'weather_1', 'weather_2']

In [5]:
dup_col = collisions[collisions.case_id.duplicated(keep=False)].sort_values('case_id')
dup_col

,case_id,col_date,col_severity,col_time,county_city,hit_run,jurisdiction,lighting,loc_type,officer_id,...,pcf_type,process_date,ramp_int,road_con_1,road_con_2,road_surf,tow_away,col_type,weather_1,weather_2
87792,97293,2002-01-19,property damage only,06:04:00,3705,not hit and run,3705.0,dark with street lights not functioning,NaN,216,...,vehicle code violation,2002-07-29,NaN,holes,construction,dry,0.0,hit object,clear,NaN
880997,97293,2003-10-28,property damage only,16:25:00,1000,not hit and run,9435.0,daylight,NaN,15986,...,vehicle code violation,2003-12-23,NaN,normal,NaN,dry,1.0,hit object,clear,NaN
344138,373108,2002-05-26,pain,10:57:00,3335,not hit and run,3300.0,daylight,highway,2474,...,vehicle code violation,2003-01-21,NaN,normal,NaN,dry,0.0,rear end,clear,NaN
3381542,373108,2004-04-11,property damage only,14:26:00,3311,not hit and run,3311.0,daylight,NaN,15147,...,vehicle code violation,2004-06-22,NaN,normal,NaN,dry,1.0,rear end,clear,NaN
874712,965874,2003-08-22,property damage only,06:20:00,2900,not hit and run,9222.0,daylight,highway,16035,...,vehicle code violation,2004-06-15,NaN,normal,NaN,dry,1.0,hit object,cloudy,NaN
3622939,965874,2005-04-11,property damage only,14:50:00,3607,not hit and run,9855.0,daylight,NaN,15364,...,vehicle code violation,2005-06-20,NaN,normal,NaN,dry,1.0,rear end,clear,NaN


We need to delete the duplicated cases which have the same ['case_id']

In [6]:
collisions = collisions.drop_duplicates(subset=['case_id'], keep=False)
collisions.shape

(3678057, 24)

In [7]:
collisions.isnull().any()

case_id                   False
col_date                  False
col_severity              False
col_time                   True
county_city               False
hit_run                   False
jurisdiction               True
lighting                   True
loc_type                   True
officer_id                 True
pcf_violation_code         True
pcf_violation_category     True
subsection                 True
population                 True
pcf_type                   True
process_date              False
ramp_int                   True
road_con_1                 True
road_con_2                 True
road_surf                  True
tow_away                   True
col_type                   True
weather_1                  True
weather_2                  True
dtype: bool

### road_en

In [8]:
road_en = collisions[['road_con_1']].append(collisions[['road_con_2']].rename(columns={'road_con_2': 'road_con_1'}), ignore_index=True)
road_en = pd.DataFrame(set(road_en.road_con_1), columns=['road_con']).dropna().sort_values('road_con').reset_index(drop=True).reset_index()
road_en.columns = ['road_num', 'road_con']
road_en 

,road_num,road_con
0,0,construction
1,1,flooded
2,2,holes
3,3,loose material
4,4,normal
5,5,obstruction
6,6,other
7,7,reduced width


In [9]:
road_en.to_csv(r'../data/road_en.csv', index=False)

### under_r

In [10]:
under_r_1 = collisions.merge(road_en.rename(columns={'road_con': 'road_con_1'}), on=['road_con_1'], how='left')
under_r_1 = under_r_1[['case_id', 'road_num']].dropna().sort_values('case_id').reset_index(drop=True)
under_r_2 = collisions.merge(road_en.rename(columns={'road_con': 'road_con_2'}), on=['road_con_2'], how='left')
under_r_2 = under_r_2[['case_id', 'road_num']].dropna().sort_values('case_id').reset_index(drop=True)
under_r = under_r_1.append(under_r_2, ignore_index=True).sort_values('case_id').reset_index(drop=True)
under_r

,case_id,road_num
0,1,4.0
1,2,4.0
2,3,4.0
3,4,4.0
4,5,4.0
...,...,...
3652139,9870011224092016011,4.0
3652140,9870011224123011850,4.0
3652141,9870011226102009803,4.0
3652142,9870011228210011458,4.0


In [11]:
under_r.to_csv(r'../data/under_r.csv', index=False)

### weather_en

In [12]:
weather_en = collisions[['weather_1']].append(collisions[['weather_2']].rename(columns={'weather_2': 'weather_1'}), ignore_index=True)
weather_en = pd.DataFrame(set(weather_en.weather_1), columns=['weather']).dropna().sort_values('weather').reset_index(drop=True).reset_index()
weather_en.columns = ['wea_num', 'weather']
weather_en 

,wea_num,weather
0,0,clear
1,1,cloudy
2,2,fog
3,3,other
4,4,raining
5,5,snowing
6,6,wind


In [13]:
weather_en.to_csv(r'../data/weather_en.csv', index=False)

### under_w

In [14]:
under_w_1 = collisions.merge(weather_en.rename(columns={'weather': 'weather_1'}), on=['weather_1'], how='left')
under_w_1 = under_w_1[['case_id', 'wea_num']].dropna().sort_values('case_id').reset_index(drop=True)
under_w_2 = collisions.merge(weather_en.rename(columns={'weather': 'weather_2'}), on=['weather_2'], how='left')
under_w_2 = under_w_2[['case_id', 'wea_num']].dropna().sort_values('case_id').reset_index(drop=True)
under_w = under_w_1.append(under_w_2, ignore_index=True).sort_values('case_id').reset_index(drop=True)
under_w

,case_id,wea_num
0,1,0.0
1,2,0.0
2,3,0.0
3,4,0.0
4,5,0.0
...,...,...
3763734,9870011224092016011,0.0
3763735,9870011224123011850,0.0
3763736,9870011226102009803,0.0
3763737,9870011228210011458,1.0


In [15]:
under_w.to_csv(r'../data/under_w.csv', index=False)

### location

In [16]:
location = collisions[['population', 'county_city','loc_type', 'ramp_int']]
location = location.drop_duplicates().reset_index(drop=True)
location = location.reset_index()
location.columns = ['loc_num', 'population', 'county_city','loc_type', 'ramp_int']
location

,loc_num,population,county_city,loc_type,ramp_int
0,0,9.0,1900,NaN,NaN
1,1,9.0,1500,NaN,NaN
2,2,6.0,1502,highway,NaN
3,3,6.0,1502,NaN,NaN
4,4,9.0,1500,highway,NaN
...,...,...,...,...,...
4357,4357,4.0,3605,highway,7.0
4358,4358,4.0,3603,highway,7.0
4359,4359,4.0,3618,highway,7.0
4360,4360,3.0,3690,highway,7.0


In [17]:
location.to_csv(r'../data/location.csv', index=False)

### condition

In [18]:
condition = collisions[['lighting', 'road_surf']]
condition = condition.drop_duplicates().reset_index(drop=True)
condition = condition.reset_index()
condition.columns = ['con_num', 'lighting', 'road_surf']
condition

,con_num,lighting,road_surf
0,0,daylight,dry
1,1,NaN,dry
2,2,dusk or dawn,dry
3,3,dark with street lights,dry
4,4,dark with no street lights,dry
5,5,daylight,wet
6,6,daylight,snowy
7,7,dark with no street lights,wet
8,8,dark with street lights,wet
9,9,daylight,NaN


In [19]:
condition.to_csv(r'../data/condition.csv', index=False)

### pcf

In [20]:
pcf = collisions[['pcf_violation_code', 'pcf_violation_category', 'subsection', 'pcf_type']]
pcf = pcf.drop_duplicates().reset_index(drop=True)
pcf = pcf.reset_index()
pcf = pcf.rename(columns={'index': 'pcf_num'})
pcf

,pcf_num,pcf_violation_code,pcf_violation_category,subsection,pcf_type
0,0,22107.0,improper turning,NaN,vehicle code violation
1,1,22515.0,hazardous parking,A,vehicle code violation
2,2,23114.0,other hazardous violation,A,vehicle code violation
3,3,22450.0,traffic signals and signs,A,vehicle code violation
4,4,22350.0,speeding,NaN,vehicle code violation
...,...,...,...,...,...
557,557,22406.0,speeding,E,vehicle code violation
558,558,22510.0,hazardous parking,NaN,vehicle code violation
559,559,36509.0,lights,B,vehicle code violation
560,560,24600.0,lights,D,vehicle code violation


In [21]:
pcf.to_csv(r'../data/pcf.csv', index=False)

### case

In [22]:
under = collisions.merge(condition, on=['lighting', 'road_surf'], how='left')
under = under[['case_id', 'con_num']]
under

,case_id,con_num
0,1,0
1,2,1
2,3,0
3,4,0
4,5,0
...,...,...
3678052,9870011224092016011,0
3678053,9870011224123011850,0
3678054,9870011226102009803,0
3678055,9870011228210011458,4


In [23]:
collide_in = collisions.merge(location, on=['population', 'county_city','loc_type', 'ramp_int'], how='left')
collide_in = collide_in[['case_id', 'loc_num']]
collide_in

,case_id,loc_num
0,1,0
1,2,1
2,3,2
3,4,1
4,5,1
...,...,...
3678052,9870011224092016011,4351
3678053,9870011224123011850,273
3678054,9870011226102009803,4352
3678055,9870011228210011458,4352


In [24]:
cause = collisions.merge(pcf, on=['pcf_violation_code', 'pcf_violation_category', 'subsection', 'pcf_type'], how='left')
cause = cause[['case_id', 'pcf_num']]
cause

,case_id,pcf_num
0,1,0
1,2,1
2,3,2
3,4,3
4,5,4
...,...,...
3678052,9870011224092016011,0
3678053,9870011224123011850,22
3678054,9870011226102009803,7
3678055,9870011228210011458,0


In [69]:
case = collisions[['case_id', 'col_date', 'col_severity', 'col_time', 'hit_run',\
                   'jurisdiction', 'officer_id', 'process_date', 'tow_away', 'col_type']]
case = case.merge(collide_in, on='case_id').merge(under, on='case_id').merge(cause, on='case_id')
case['col_time'] = case['col_date'] + ' ' + case['col_time']
case

,case_id,col_date,col_severity,col_time,hit_run,jurisdiction,officer_id,process_date,tow_away,col_type,loc_num,con_num,pcf_num
0,1,2002-01-18,property damage only,2002-01-18 15:30:00,not hit and run,9575.0,16418,2002-06-14,1.0,rear end,0,0,0
1,2,2002-02-13,property damage only,2002-02-13 19:30:00,misdemeanor,9420.0,14360,2002-03-29,0.0,hit object,1,1,1
2,3,2002-02-11,property damage only,2002-02-11 15:30:00,misdemeanor,9420.0,16117,2003-07-15,0.0,hit object,2,0,2
3,4,2002-02-12,property damage only,2002-02-12 07:45:00,misdemeanor,9420.0,6078,2002-05-22,0.0,sideswipe,1,0,3
4,5,2002-02-14,severe injury,2002-02-14 11:35:00,not hit and run,9420.0,13851,2003-07-28,1.0,overturned,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3678052,9870011224092016011,2001-12-24,pain,2001-12-24 09:20:00,not hit and run,9870.0,16011,2002-04-26,1.0,hit object,4351,0,0
3678053,9870011224123011850,2001-12-24,property damage only,2001-12-24 12:30:00,not hit and run,9870.0,11850,2002-04-26,0.0,sideswipe,273,0,22
3678054,9870011226102009803,2001-12-26,other injury,2001-12-26 10:20:00,not hit and run,9870.0,9803,2002-03-29,1.0,head-on,4352,0,7
3678055,9870011228210011458,2001-12-28,property damage only,2001-12-28 21:00:00,misdemeanor,9870.0,11458,2002-03-08,0.0,hit object,4352,4,0


In [70]:
case.to_csv(r'../data/case.csv', index=False)

# parties.csv

In [27]:
parties = pd.read_csv(data_folder+'parties2018.csv', error_bad_lines=False, low_memory=False)

In [28]:
parties.head()

,at_fault,case_id,cellphone_use,financial_responsibility,hazardous_materials,id,movement_preceding_collision,other_associate_factor_1,other_associate_factor_2,party_age,...,party_number,party_safety_equipment_1,party_safety_equipment_2,party_sex,party_sobriety,party_type,school_bus_related,statewide_vehicle_type,vehicle_make,vehicle_year
0,1,1,C,Y,NaN,8,other,L,NaN,84.0,...,1,G,NaN,female,A,driver,NaN,passenger car,FORD,2000.0
1,0,1,D,O,NaN,9,parked,N,NaN,NaN,...,2,NaN,NaN,NaN,H,parked vehicle,NaN,passenger car,BUICK,1992.0
2,0,2,NaN,O,NaN,10,proceeding straight,O,NaN,NaN,...,1,NaN,NaN,NaN,H,other,NaN,pickup or panel truck,TOYOTA,NaN
3,0,3,D,Y,NaN,11,proceeding straight,N,NaN,20.0,...,1,G,NaN,female,A,driver,NaN,passenger car,FORD,1995.0
4,1,3,D,NaN,NaN,12,proceeding straight,N,NaN,NaN,...,2,NaN,NaN,NaN,G,driver,NaN,pickup or panel truck,NaN,NaN


In [29]:
parties.isnull().any()

at_fault                        False
case_id                         False
cellphone_use                    True
financial_responsibility         True
hazardous_materials              True
id                              False
movement_preceding_collision     True
other_associate_factor_1         True
other_associate_factor_2         True
party_age                        True
party_drug_physical              True
party_number                    False
party_safety_equipment_1         True
party_safety_equipment_2         True
party_sex                        True
party_sobriety                   True
party_type                       True
school_bus_related               True
statewide_vehicle_type           True
vehicle_make                     True
vehicle_year                     True
dtype: bool

In [30]:
len(parties)

7286606

In [31]:
len(parties['id'].drop_duplicates())

7286606

We need to delete the duplicated cases which have the same ['case_id'] in 'collisions' dataframe.

In [32]:
dup_c = dup_col[['case_id']].drop_duplicates().reset_index(drop=True)
one_c = pd.DataFrame(np.ones(len(dup_c)),columns=['du'])
dup_c['du'] = one_c['du']
dup_c

,case_id,du
0,97293,1.0
1,373108,1.0
2,965874,1.0


In [33]:
df_party = parties.merge(dup_c, on=['case_id'], how='left')
df_party = df_party[df_party.du!=1].drop('du', axis=1)
len(parties), len(df_party)

(7286606, 7286596)

In [34]:
df_party[df_party[['case_id','party_number']].duplicated()]

,at_fault,case_id,cellphone_use,financial_responsibility,hazardous_materials,id,movement_preceding_collision,other_associate_factor_1,other_associate_factor_2,party_age,...,party_number,party_safety_equipment_1,party_safety_equipment_2,party_sex,party_sobriety,party_type,school_bus_related,statewide_vehicle_type,vehicle_make,vehicle_year


Therefore, every ['case_id', 'party_num'] corresponds to an unique party.

In [35]:
df_party.head()

,at_fault,case_id,cellphone_use,financial_responsibility,hazardous_materials,id,movement_preceding_collision,other_associate_factor_1,other_associate_factor_2,party_age,...,party_number,party_safety_equipment_1,party_safety_equipment_2,party_sex,party_sobriety,party_type,school_bus_related,statewide_vehicle_type,vehicle_make,vehicle_year
0,1,1,C,Y,NaN,8,other,L,NaN,84.0,...,1,G,NaN,female,A,driver,NaN,passenger car,FORD,2000.0
1,0,1,D,O,NaN,9,parked,N,NaN,NaN,...,2,NaN,NaN,NaN,H,parked vehicle,NaN,passenger car,BUICK,1992.0
2,0,2,NaN,O,NaN,10,proceeding straight,O,NaN,NaN,...,1,NaN,NaN,NaN,H,other,NaN,pickup or panel truck,TOYOTA,NaN
3,0,3,D,Y,NaN,11,proceeding straight,N,NaN,20.0,...,1,G,NaN,female,A,driver,NaN,passenger car,FORD,1995.0
4,1,3,D,NaN,NaN,12,proceeding straight,N,NaN,NaN,...,2,NaN,NaN,NaN,G,driver,NaN,pickup or panel truck,NaN,NaN


In [36]:
df_party.columns

Index(['at_fault', 'case_id', 'cellphone_use', 'financial_responsibility',
       'hazardous_materials', 'id', 'movement_preceding_collision',
       'other_associate_factor_1', 'other_associate_factor_2', 'party_age',
       'party_drug_physical', 'party_number', 'party_safety_equipment_1',
       'party_safety_equipment_2', 'party_sex', 'party_sobriety', 'party_type',
       'school_bus_related', 'statewide_vehicle_type', 'vehicle_make',
       'vehicle_year'],
      dtype='object')

In [37]:
df_party.columns = ['at_fault', 'case_id', 'phone', 'fin_resp',
       'haz_mat', 'party_id', 'move_pre','other_fac_1', 'other_fac_2', 
       'age','drug_phy', 'party_num', 'safety_equip_1',
       'safety_equip_2', 'sex', 'sobriety', 'party_type',
       'school_bus_rel', 've_type', 've_make', 've_year']

### party

In [38]:
party = df_party[['party_id', 'case_id', 'at_fault', 'phone', 'fin_resp',
       'haz_mat', 'move_pre', 'age','drug_phy', 'party_num', 'sex', 'sobriety', 'party_type']]
party = party.drop_duplicates().reset_index(drop=True)
party

,party_id,case_id,at_fault,phone,fin_resp,haz_mat,move_pre,age,drug_phy,party_num,sex,sobriety,party_type
0,8,1,1,C,Y,NaN,other,84.0,NaN,1,female,A,driver
1,9,1,0,D,O,NaN,parked,NaN,H,2,NaN,H,parked vehicle
2,10,2,0,NaN,O,NaN,proceeding straight,NaN,H,1,NaN,H,other
3,11,3,0,D,Y,NaN,proceeding straight,20.0,NaN,1,female,A,driver
4,12,3,1,D,NaN,NaN,proceeding straight,NaN,G,2,NaN,G,driver
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7286591,16432239,9870011226102009803,1,D,Y,NaN,making left turn,66.0,NaN,1,male,A,driver
7286592,16432240,9870011226102009803,0,D,Y,NaN,proceeding straight,20.0,NaN,2,female,A,driver
7286593,16432241,9870011228210011458,1,D,N,NaN,ran off road,NaN,NaN,1,NaN,G,driver
7286594,16432242,9870011231152508671,1,NaN,Y,NaN,passing other vehicle,19.0,NaN,1,female,A,driver


In [63]:
party.to_csv(r'../data/party_involve.csv', index=False)

### other_fac_en

In [40]:
other_fac_en = df_party[['other_fac_1']].append(df_party[['other_fac_2']].rename(columns={'other_fac_2': 'other_fac_1'}), ignore_index=True)
other_fac_en = pd.DataFrame(set(other_fac_en.other_fac_1), columns=['other_fac']).dropna().sort_values('other_fac').reset_index(drop=True).reset_index()
other_fac_en.columns = ['other_fac_num', 'other_fac']
other_fac_en 

,other_fac_num,other_fac
0,0,A
1,1,E
2,2,F
3,3,G
4,4,H
5,5,I
6,6,J
7,7,K
8,8,L
9,9,M


In [41]:
other_fac_en.to_csv(r'../data/other_fac_en.csv', index=False)

### have

In [42]:
have_1 = df_party.merge(other_fac_en.rename(columns={'other_fac': 'other_fac_1'}), on=['other_fac_1'], how='left')
have_1 = have_1[['case_id', 'other_fac_num']].dropna().sort_values('case_id').reset_index(drop=True)
have_2 = df_party.merge(other_fac_en.rename(columns={'other_fac': 'other_fac_2'}), on=['other_fac_2'], how='left')
have_2 = have_2[['case_id', 'other_fac_num']].dropna().sort_values('case_id').reset_index(drop=True)
have = have_1.append(have_2, ignore_index=True).sort_values('case_id').reset_index(drop=True)
have

,case_id,other_fac_num
0,1,8.0
1,1,10.0
2,2,11.0
3,3,10.0
4,3,10.0
...,...,...
6949298,9870011226102009803,10.0
6949299,9870011228210011458,10.0
6949300,9870011231152508671,0.0
6949301,9870011231152508671,0.0


In [43]:
have.to_csv(r'../data/have.csv', index=False)

### vehicle

In [44]:
vehicle = df_party[['ve_type', 've_make', 've_year']].drop_duplicates().reset_index(drop=True)\
                .reset_index().rename(columns={'index': 've_num'})
vehicle

,ve_num,ve_type,ve_make,ve_year
0,0,passenger car,FORD,2000.0
1,1,passenger car,BUICK,1992.0
2,2,pickup or panel truck,TOYOTA,NaN
3,3,passenger car,FORD,1995.0
4,4,pickup or panel truck,NaN,NaN
...,...,...,...,...
24094,24094,truck or truck tractor,MISCELLANEOUS,1937.0
24095,24095,pickup or panel truck,PORSCHE,1986.0
24096,24096,emergency vehicle,AMERICAN MOTORS,1997.0
24097,24097,passenger car,OTHER FOREIGN,1926.0


In [45]:
vehicle.to_csv(r'../data/vehicle.csv', index=False)

### take

In [46]:
take = df_party.merge(vehicle, on=['ve_type', 've_make', 've_year'], how='left')
take = take[['party_id', 've_num', 'school_bus_rel']]
take

,party_id,ve_num,school_bus_rel
0,8,0,NaN
1,9,1,NaN
2,10,2,NaN
3,11,3,NaN
4,12,4,NaN
...,...,...,...
7286591,16432239,668,NaN
7286592,16432240,16,NaN
7286593,16432241,296,NaN
7286594,16432242,640,NaN


In [47]:
take.to_csv(r'../data/take.csv', index=False)

### safety_equip_en

In [48]:
safety_equip_en = df_party[['safety_equip_1']].append(df_party[['safety_equip_2']].rename(columns={'safety_equip_2': 'safety_equip_1'}), ignore_index=True)
safety_equip_en = pd.DataFrame(set(safety_equip_en.safety_equip_1), columns=['safety_equip']).dropna().sort_values('safety_equip').reset_index(drop=True).reset_index()
safety_equip_en.columns = ['safety_equip_num', 'safety_equip']
safety_equip_en 

,safety_equip_num,safety_equip
0,0,A
1,1,B
2,2,C
3,3,D
4,4,E
5,5,F
6,6,G
7,7,H
8,8,J
9,9,K


In [49]:
safety_equip_en.to_csv(r'../data/safety_equip_en.csv', index=False)

### have_ps

In [50]:
have_ps_1 = df_party.merge(safety_equip_en.rename(columns={'safety_equip': 'safety_equip_1'}), on=['safety_equip_1'], how='left')
have_ps_1 = have_ps_1[['party_id', 'safety_equip_num']].dropna().sort_values('party_id').reset_index(drop=True)
have_ps_2 = df_party.merge(safety_equip_en.rename(columns={'safety_equip': 'safety_equip_2'}), on=['safety_equip_2'], how='left')
have_ps_2 = have_ps_2[['party_id', 'safety_equip_num']].dropna().sort_values('party_id').reset_index(drop=True)
have_ps = have_ps_1.append(have_ps_2, ignore_index=True).sort_values('party_id').reset_index(drop=True)
have_ps

,party_id,safety_equip_num
0,5,22.0
1,8,6.0
2,11,6.0
3,13,1.0
4,14,6.0
...,...,...
8751338,16432239,6.0
8751339,16432240,6.0
8751340,16432241,1.0
8751341,16432242,6.0


In [51]:
have_ps.to_csv(r'../data/have_ps.csv', index=False)

# victims.csv

In [52]:
victims = pd.read_csv(data_folder+'victims2018.csv', error_bad_lines=False, low_memory=False)

In [53]:
victims.head()

,case_id,id,party_number,victim_age,victim_degree_of_injury,victim_ejected,victim_role,victim_safety_equipment_1,victim_safety_equipment_2,victim_seating_position,victim_sex
0,3,3,1,21.0,no injury,0.0,2,G,NaN,3.0,male
1,5,4,1,44.0,severe injury,0.0,1,G,NaN,1.0,male
2,8,5,1,59.0,no injury,0.0,2,G,NaN,3.0,female
3,8,6,2,31.0,no injury,0.0,2,G,NaN,3.0,male
4,9,7,2,14.0,complaint of pain,0.0,2,C,NaN,6.0,female


In [54]:
victims.isnull().any()

case_id                      False
id                           False
party_number                 False
victim_age                    True
victim_degree_of_injury      False
victim_ejected                True
victim_role                  False
victim_safety_equipment_1     True
victim_safety_equipment_2     True
victim_seating_position       True
victim_sex                    True
dtype: bool

Delete aforementioned duplicated cases

In [55]:
df_victims = victims.merge(dup_c, on=['case_id'], how='left')
df_victims = df_victims[df_victims.du!=1].drop('du', axis=1)
len(victims), len(df_victims)

(4082685, 4082679)

In [56]:
df_victims.head()

,case_id,id,party_number,victim_age,victim_degree_of_injury,victim_ejected,victim_role,victim_safety_equipment_1,victim_safety_equipment_2,victim_seating_position,victim_sex
0,3,3,1,21.0,no injury,0.0,2,G,NaN,3.0,male
1,5,4,1,44.0,severe injury,0.0,1,G,NaN,1.0,male
2,8,5,1,59.0,no injury,0.0,2,G,NaN,3.0,female
3,8,6,2,31.0,no injury,0.0,2,G,NaN,3.0,male
4,9,7,2,14.0,complaint of pain,0.0,2,C,NaN,6.0,female


In [57]:
df_victims.columns = ['case_id', 'vic_id', 'party_number', 'vic_age','deg_injury', 'ejected', 
                      'vic_role','safety_equip_1', 'safety_equip_2','vic_seat', 'vic_sex']

### victim

In [58]:
associate = df_victims[['case_id', 'party_number','vic_id']]
associate = associate.drop_duplicates().reset_index(drop=True)
associate

,case_id,party_number,vic_id
0,3,1,3
1,5,1,4
2,8,1,5
3,8,2,6
4,9,2,7
...,...,...,...
4082674,9870011226102009803,2,8817531
4082675,9870011231152508671,1,8817532
4082676,9870011231152508671,2,8817533
4082677,9870011231152508671,2,8817534


In [59]:
victim = df_victims[['vic_id', 'vic_age', 'deg_injury', 'ejected', 'vic_role', 'vic_seat', 'vic_sex']]
victim = victim.merge(associate, on=['vic_id'])
victim

,vic_id,vic_age,deg_injury,ejected,vic_role,vic_seat,vic_sex,case_id,party_number
0,3,21.0,no injury,0.0,2,3.0,male,3,1
1,4,44.0,severe injury,0.0,1,1.0,male,5,1
2,5,59.0,no injury,0.0,2,3.0,female,8,1
3,6,31.0,no injury,0.0,2,3.0,male,8,2
4,7,14.0,complaint of pain,0.0,2,6.0,female,9,2
...,...,...,...,...,...,...,...,...,...
4082674,8817531,20.0,other visible injury,0.0,1,1.0,female,9870011226102009803,2
4082675,8817532,27.0,complaint of pain,0.0,2,3.0,female,9870011231152508671,1
4082676,8817533,74.0,complaint of pain,0.0,2,3.0,female,9870011231152508671,2
4082677,8817534,64.0,complaint of pain,0.0,1,1.0,male,9870011231152508671,2


In [64]:
victim.to_csv(r'../data/associate_victim.csv', index=False)

### have_vs

In [61]:
have_vs_1 = df_victims.merge(safety_equip_en.rename(columns={'safety_equip': 'safety_equip_1'}), on=['safety_equip_1'], how='left')
have_vs_1 = have_vs_1[['vic_id', 'safety_equip_num']].dropna().sort_values('vic_id').reset_index(drop=True)
have_vs_2 = df_victims.merge(safety_equip_en.rename(columns={'safety_equip': 'safety_equip_2'}), on=['safety_equip_2'], how='left')
have_vs_2 = have_vs_2[['vic_id', 'safety_equip_num']].dropna().sort_values('vic_id').reset_index(drop=True)
have_vs = have_vs_1.append(have_vs_2, ignore_index=True).sort_values('vic_id').reset_index(drop=True)
have_vs

,vic_id,safety_equip_num
0,3,6.0
1,4,6.0
2,5,6.0
3,6,6.0
4,7,2.0
...,...,...
5520437,8817531,6.0
5520438,8817532,7.0
5520439,8817533,6.0
5520440,8817534,6.0


In [62]:
have_vs.to_csv(r'../data/have_vs.csv', index=False)